In [1]:
import prior

dataset = prior.load_dataset("procthor-10k")
dataset

Fetching reference HEAD


[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 24739.03it/s]


DatasetDict(
    train=Dataset(
    dataset=procthor-dataset,
    size=10000,
    split=train
),
    val=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=val
),
    test=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=test
)
)

In [2]:
import numpy as np
def teleport(controller, target=None):
    event = controller.step("GetReachablePositions")
    reachable_positions = event.metadata["actionReturn"]
    # Pick a random target
    if target is None:
        target = np.random.choice(reachable_positions)

    event = controller.step(
        action="TeleportFull",
        x=target["x"],
        y=target["y"],
        z=target["z"],
        rotation={"x": 0, "y": 0, "z": 0},
        horizon=0,
        standing=True
    )

    return event


In [3]:
from rl import PPO, ActorCritic, Env, RolloutBuffer, ClipEnvNoPenalty, CLIPNovelty
from models import FrozenResNetEncoder, SlidingWindowTransformerActor, ConvDepthEncoder, SlidingWindowTransformerCritic
from cons import MINIBATCHES, EPISODE_STEPS, FEAT_DIM, NUM_ACTIONS, DEVICE

In [4]:
import torch
import matplotlib.pyplot as plt
from rl import save_actor_critic
import numpy as np
from cons import NUM_ACTIONS, EPISODE_STEPS, MINIBATCHES, DEVICE

def train(controller, name: str, ppo: PPO, env: Env, actor_critic: ActorCritic, total_updates=10):

    event = controller.step("Pass")
    rewards = []
    episode_rewards = []

    for upd in range(total_updates):
        buf = RolloutBuffer()

        for mb in range(MINIBATCHES):

            episode_seq = []
            episode_reward = 0.0
            actions_seq = []

            for t in range(1, EPISODE_STEPS + 1):

                # =============================
                # RAW RGB
                # =============================
                rgb_frame = ppo.obs_from_event(event.frame)  # (3,H,W)

                # =============================
                # RAW DEPTH  (normalized)
                # =============================
                depth_tensor = ppo.obs_from_event(event.depth_frame) / 10.

                # =============================
                # Encode for policy (no grad)
                # =============================
                with torch.no_grad():
                    rgb_input = rgb_frame.unsqueeze(0).unsqueeze(0)     # (1,1,3,H,W)
                    depth_input = depth_tensor.unsqueeze(0).unsqueeze(0)  # (1,1,1,H,W)

                    rgb_embed = actor_critic.rgb_encoder(rgb_input).squeeze(0).squeeze(0)
                    depth_embed = actor_critic.depth_encoder(depth_input).squeeze(0).squeeze(0)

                    fused_embed = torch.cat([rgb_embed, depth_embed], dim=-1)

                # Build sequence for transformer
                episode_seq.append(fused_embed)
                obs_seq = torch.stack(episode_seq, dim=0).unsqueeze(0).to(DEVICE)

                # =============================
                # Action history
                # =============================
                if len(actions_seq) == 0:
                    actions_seq.append(torch.randint(0, NUM_ACTIONS, ()).item())

                actions_tensor = torch.tensor(actions_seq, dtype=torch.long, device=DEVICE).unsqueeze(0)

                # =============================
                # Policy + Value
                # =============================
                logits, value = ppo.act_and_value(obs_seq, actions_tensor, actor_critic)
                dist = torch.distributions.Categorical(logits=logits)
                action_idx = dist.sample().item()
                logp = dist.log_prob(torch.tensor(action_idx, device=DEVICE)).item()

                # =============================
                # Step environment
                # =============================
                event, reward = env.step_env(controller, action_idx)
                done = (t == EPISODE_STEPS)

                # =============================
                # Store RAW frames ONLY
                # =============================
                buf.add(
                    rgb_frame=rgb_frame.cpu(),
                    depth_frame=depth_tensor.cpu(),
                    action_idx=action_idx,
                    logp=logp,
                    reward=reward,
                    value=value,
                    done=done
                )

                rewards.append(reward)
                episode_reward += reward / EPISODE_STEPS
                actions_seq.append(action_idx)

                # =============================
                # Reset episode if done
                # =============================
                if done:
                    env.reset()
                    if np.random.rand() > 0.5:
                        event = teleport(controller)

            episode_rewards.append(episode_reward)

        # =============================
        # PPO UPDATE (re-encodes raw)
        # =============================
        ppo.ppo_update(buf, actor_critic)
        save_actor_critic(actor_critic, name)

        print(f"Update {upd+1}/{total_updates} — steps: {len(buf)}")

        torch.cuda.empty_cache()  # optional safety

    return buf, rewards, episode_rewards



In [5]:
ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetEncoder()
depth_encoder = ConvDepthEncoder()
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS)
critic = SlidingWindowTransformerCritic(FEAT_DIM)
clip_novelty = CLIPNovelty()
clip_env = ClipEnvNoPenalty(clip_novelty)
clip_actor_critic = ActorCritic(encoder, depth_encoder, actor, critic)


In [6]:
from rl import load_actor_critic

load_actor_critic(clip_actor_critic, "data/weight_long.pt", device=DEVICE)

[🔁] Actor-Critic checkpoint loaded from data/weight_long.pt


In [7]:
from ai2thor.controller import Controller

NUM_ENV = 50
for i in range(NUM_ENV):
    try:
        rand_env = torch.randint(0, len(dataset["train"]), (1,)).item()
        house = dataset["train"][rand_env]
        controller = Controller(scene=house, snapToGrid=False, rotateStepDegrees=30, renderDepthImage=True)
        teleport(controller)
        buf, rewards, episode_rewards = train(controller, "weight.pt", ppo, clip_env, clip_actor_critic, 5)
        plt.plot(rewards)
        plt.show()
        plt.plot(episode_rewards)
        plt.show()
    finally:
        controller.stop()


KeyboardInterrupt: 

In [62]:
from cons import ACTIONS
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import imageio


def inference(get_distribution: Callable[[torch.Tensor, ActorCritic], torch.distributions.Categorical], init_position: dict[str, float], env: Env, actor_critic: ActorCritic, plot=True):
    n = 256
    n_row = 32
    positions = []

    plt.figure(figsize=(n // n_row * 2, n_row * 2))
    event = teleport(controller, init_position)
    episode_seq = deque(maxlen=EPISODE_STEPS)
    actions_seq = deque(maxlen=EPISODE_STEPS)
    raw_obs = []
    for t in range(1, n + 1):
        positions.append([event.metadata["agent"]["position"]["x"], event.metadata["agent"]["position"]["z"]])
        with torch.no_grad():
            obs_t = ppo.obs_from_event(event)  # (C,H,W)
            obs_t_encoded = actor_critic.actor_critic_encoder(obs_t.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0)
            obs_seq = torch.stack(list(episode_seq) + [obs_t_encoded], dim=0).unsqueeze(0).to(device=DEVICE)

        if len(actions_seq) == 0:
            actions_seq.append(torch.randint(0, NUM_ACTIONS, (1, 1)).item())
        
        actions_tensor = torch.tensor(actions_seq, dtype=torch.long, device=DEVICE).unsqueeze(0)
        dist = get_distribution(obs_seq, actions_tensor, actor_critic)
        action_idx = dist.sample()
        logp = dist.log_prob(action_idx)
        
        action_idx, logp = action_idx.item(), logp.item()
        event, reward = env.step_env(controller, action_idx)

        # store one step
        episode_seq.append(obs_t_encoded)
        actions_seq.append(action_idx)
        raw_obs.append(obs_t)
        
        if plot:
            # Plot frame and action
            plt.subplot(n_row, n // n_row, t)
            plt.title("action: " + ACTIONS[action_idx] + ", r: " + f"{reward:.2f}" + "\n, prob = " + f"{torch.exp(dist.log_prob(torch.tensor([0, 1, 2], device=DEVICE))).cpu().numpy()}", fontsize=5)
            plt.axis(False)
            plt.imshow(event.frame)
    if plot:
        plt.tight_layout()
        plt.show()
        
        # ---- Plot 2D trajectory of the agent ----
        positions = np.array(positions)
        plt.figure(figsize=(4, 4))
        plt.plot(positions[:, 0], positions[:, 1], "-o", markersize=3)
        plt.xlabel("x")
        plt.ylabel("z")
        plt.title("Agent trajectory over n steps")
        plt.grid(True)
        plt.show()

    return raw_obs

def inference_video_mp4(
    get_distribution,
    init_position: dict[str, float],
    env: Env,
    actor_critic: ActorCritic,
    video_path="rollout.mp4",
    fps=10,
    n_steps=256
):
    episode_seq = deque(maxlen=EPISODE_STEPS)
    actions_seq = deque(maxlen=EPISODE_STEPS)

    writer = imageio.get_writer(video_path, fps=fps)

    # start episode
    event = teleport(controller, init_position)
    positions = []

    for t in range(1, n_steps + 1):

        # ---- Add current frame to video ----
        writer.append_data(event.frame[:, :, ::-1])   # convert RGB→BGR if needed

        # track positions
        positions.append([
            event.metadata["agent"]["position"]["x"],
            event.metadata["agent"]["position"]["z"],
        ])

        with torch.no_grad():
            obs_t = ppo.obs_from_event(event)  
            obs_enc = actor_critic.actor_critic_encoder(
                obs_t.unsqueeze(0).unsqueeze(0)
            ).squeeze(0).squeeze(0)

            obs_seq = torch.stack(
                list(episode_seq) + [obs_enc], dim=0
            ).unsqueeze(0).to(DEVICE)

        # ---- Action ----
        if len(actions_seq) == 0:
            actions_seq.append(torch.randint(0, NUM_ACTIONS, ()).item())

        actions_tensor = torch.tensor(
            actions_seq, dtype=torch.long, device=DEVICE
        ).unsqueeze(0)

        dist = get_distribution(obs_seq, actions_tensor, actor_critic)
        action_idx = dist.sample().item()

        # ---- Step env ----
        event, reward = env.step_env(controller, action_idx)

        # ---- Store ----
        episode_seq.append(obs_enc)
        actions_seq.append(action_idx)

    writer.close()
    print(f"[🎞️] Saved video to {video_path}")

    return positions


def get_distributions(obs_seq, actions_tensor, clip_actor_critic):
    logits, value = ppo.act_and_value(obs_seq, actions_tensor, clip_actor_critic)
    dist = torch.distributions.Categorical(logits=logits)
    return dist

event = teleport(controller)
init_pos = event.metadata["agent"]["position"]
inference_video_mp4(get_distributions, init_pos, clip_env, clip_actor_critic)

[🎞️] Saved video to rollout.mp4


[[1.0985212326049805, 1.6973028182983398],
 [1.0985212326049805, 1.9473028182983398],
 [1.0985212326049805, 2.19730281829834],
 [1.0985212326049805, 2.44730281829834],
 [1.0985212326049805, 2.44730281829834],
 [0.9735212326049805, 2.663809061050415],
 [0.9735212326049805, 2.663809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 2.788809061050415],
 [0.7570148706436157, 3.038809061050415],
 [0.7570148706436157, 3.038809061050415],
 [0.8820148706436157, 3.2553153038024902],
 [0.8820148706436157, 3.2553153038024902],
 [1.0985212326049805, 3.3803153038024902],
 [1.0985212326049805, 3.3803153038024902],
 [1.0985212326049805, 3.3803153

In [34]:
!pip install imageio imageio-ffmpeg

In [22]:
import clip

model, preprocess = clip.load("ViT-B/32", device=DEVICE)

In [23]:
import torch
import torch.nn.functional as F
import random

def preprocess_clip(x):
    # Resize to CLIP resolution
    x = F.interpolate(x, size=(224, 224), mode="bilinear", align_corners=False)
    # Normalize (CLIP standard mean/std)
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=x.device).view(1, 3, 1, 1)
    std  = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=x.device).view(1, 3, 1, 1)
    return (x - mean) / std

def get_average(input_img):
    # ---- 1. Sample 50 random images ----
    n = min(50, len(input_img))
    indices = random.sample(range(len(input_img)), n)
    imgs = torch.stack([input_img[i] for i in indices]).to(DEVICE)  # (50, 3, H, W)

    # ---- 2. Preprocess for CLIP ----
    imgs_clip = preprocess_clip(imgs).half()  # normalized, resized

    # ---- 3. Encode with CLIP visual encoder ----
    with torch.no_grad():
        embeds = model.visual(imgs_clip)       # (50, D)
        embeds = embeds / embeds.norm(dim=-1, keepdim=True)  # normalize to unit sphere

    # ---- 4. Compute cosine similarity ----
    sim_matrix = embeds @ embeds.T             # (50, 50)
    # Optional: remove self-similarity (diagonal = 1)
    sim_matrix.fill_diagonal_(0)

    # ---- 5. Summary statistics ----
    mean_sim = sim_matrix.mean().item()

    return mean_sim

for i in range(5):

    event = teleport(controller)
    init_pos = event.metadata["agent"]["position"]
    
    def get_distributions(obs_seq, actions_tensor, clip_actor_critic):
        logits, value = ppo.act_and_value(obs_seq, actions_tensor, clip_actor_critic)
        dist = torch.distributions.Categorical(logits=logits)
        return dist

    policy = inference(get_distributions, init_pos, clip_env, clip_actor_critic, False)

    def get_distributions(obs_seq, actions_tensor, clip_actor_critic):
        dist = torch.distributions.Categorical(probs=torch.tensor([0.5, 0.25, 0.25], device=DEVICE))
        return dist

    rand_policy = inference(get_distributions, init_pos, clip_env, clip_actor_critic, False)

    mean_policies = []
    mean_rands = []
    for i in range(10):
        mean_policy = get_average(policy)
        mean_rand = get_average(rand_policy)
        mean_policies.append(mean_policy)
        mean_rands.append(mean_rand)
    
    print("Policy: " + str(np.mean(np.array(mean_policies))))
    print("Rand: " + str(np.mean(np.array(mean_rands))))



Policy: 0.877001953125
Rand: 0.86337890625
Policy: 0.87021484375
Rand: 0.8068359375
Policy: 0.83388671875
Rand: 0.845361328125
Policy: 0.8791015625
Rand: 0.876318359375
Policy: 0.80693359375
Rand: 0.79033203125


<Figure size 1600x6400 with 0 Axes>

<Figure size 1600x6400 with 0 Axes>

<Figure size 1600x6400 with 0 Axes>

<Figure size 1600x6400 with 0 Axes>

...

<Figure size 1600x6400 with 0 Axes>

<Figure size 1600x6400 with 0 Axes>

<Figure size 1600x6400 with 0 Axes>

<Figure size 1600x6400 with 0 Axes>

<Figure size 1600x6400 with 0 Axes>